# Setup Notebook

This is a setup notebook to set up resources needed for the `getting-started` notebooks.

This notebook will:
- Install required dependencies
- Configure AWS region and S3 bucket
- Create S3 bucket and upload sample assets
- Store configuration variables for other notebooks

In [ ]:
!pip install -r requirements.txt

## S3 Bucket Setup

Create a unique S3 bucket name and store it for use in other notebooks.

In [ ]:
import boto3
import json

# Get AWS account ID
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
print(f"AWS Account ID: {account_id}")

# Create unique S3 bucket name
s3_bucket = f"nova-mme-get-started-{account_id}"
print(f"S3 Bucket Name: {s3_bucket}")

In [ ]:
# Store bucket name in a config file for other notebooks
config = {
    "s3_bucket": s3_bucket,
    "account_id": account_id
}

with open('config.json', 'w') as f:
    json.dump(config, f, indent=2)

print(f"Configuration saved to config.json")
print(f"S3 bucket name: {s3_bucket}")

In [ ]:
# Create S3 bucket
s3 = boto3.client('s3')

try:
    s3.create_bucket(Bucket=s3_bucket)
    print(f"Created S3 bucket: {s3_bucket}")
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f"S3 bucket {s3_bucket} already exists")
except Exception as e:
    print(f"Error creating bucket: {e}")

In [ ]:
import os

# Upload sample assets to S3 bucket
sample_assets_dir = 'sample_assets'

for root, dirs, files in os.walk(sample_assets_dir):
    for file in files:
        local_path = os.path.join(root, file)
        s3_key = local_path.replace('\\', '/')
        
        try:
            s3.upload_file(local_path, s3_bucket, s3_key)
            print(f"Uploaded: {local_path} -> s3://{s3_bucket}/{s3_key}")
        except Exception as e:
            print(f"Error uploading {local_path}: {e}")

print("Sample assets upload complete!")

In [ ]:
region_name = 'us-east-1'
os.environ['AWS_DEFAULT_REGION'] = region_name

In [ ]:
# Store the bucket name for use in other notebooks
%store s3_bucket
%store region_name